### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
33,42645312,53662,79085,51530,0,0,1137,5,2025-04-24 19:35,2025-04-24 19:35,0,3,1968,3,1
34,42641480,103165,79085,3,0,0,1137,6,2025-04-26 01:29,2025-04-26 01:29,0,0,8100,7,1
35,42642559,149122,79085,51530,0,0,1138,0,2025-04-26 22:00,2025-04-26 22:00,0,1,556,2,1
36,42410355,122169,79085,1349,11,70,1138,1,2025-04-27 21:59,2025-04-27 21:59,3,3,7772,2,1
37,42642560,149122,79085,51530,0,0,1138,2,2025-04-28 22:00,2025-04-28 22:00,0,1,628,4,1
38,42645314,78043,79085,51531,0,0,1138,3,2025-04-29 11:00,2025-04-29 11:00,1,0,6335,3,1
39,42410356,79085,122169,1349,12,70,1138,4,2025-04-30 21:59,2025-04-30 21:59,0,3,6831,3,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
#df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

7

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,53662,38932,32769,0,5,2,1,0,Inzaghi,5,6,5,42645312
1,79085,31153,37316,1,9,1,1,0,3-4-3BC,6,8,8,42645312
0,103165,23798,36219,0,5,2,1,0,predicando,8,6,7,42641480
1,79085,46605,34184,1,2,1,0,0,3-4-3BC,6,8,9,42641480
0,149122,33668,36969,1,5,2,0,0,leva,9,8,11,42642559
1,79085,36850,33549,0,7,4,1,0,5-3-2a,10,9,10,42642559
0,122169,25487,35110,0,12,2,1,0,Nó Tático,10,7,10,42410355
1,79085,44587,34964,2,9,2,0,0,5-3-2a,10,9,10,42410355
0,149122,42535,32070,0,3,4,1,0,lobo_att,8,7,11,42642560
1,79085,27863,38328,2,8,3,2,0,5-2-3f,9,10,11,42642560


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

1714

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 1349

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,10520,12,34,11,1,0,54,6,34548054011011,1349
1,23775,12,30,10,0,2,37,10,30527037010001,1349
2,124139,12,25,8,1,3,30,11,25519030008006,1349
3,23755,12,23,7,2,3,41,18,23523041007010,1349
4,78671,12,20,6,2,4,25,21,20504025006008,1349
5,122169,12,19,5,4,3,26,16,19510026005005,1349
6,79866,12,18,5,3,4,30,13,18517030005009,1349
7,79085,12,17,5,2,5,24,17,17507024005004,1349
8,89569,12,12,4,0,8,17,25,12492017004007,1349
9,79665,12,6,2,0,10,12,67,6445012002002,1349


In [14]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [15]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [16]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [17]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [18]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

4

In [19]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()